In [5]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import glob,os
import numpy as np
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.moead import MOEAD
from pymoo.problems import *
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
from pymoo.util.ref_dirs import get_reference_directions

In [7]:
request_count = [10,20,30]
tasks = []
for file in glob.glob(os.path.join('./pymoo/problems/nfv/sfc/data/input', '*_network.txt')):

    n = file_to_network(file)
    # requests = file_to_requests(os.path.join(args.input, f'{dat}_requests.txt'))
    for cnt in request_count:
        req_file = file.replace('network', f'{cnt}requests')
        name = req_file.split('\\')[-1][:-5]
        requests = file_to_requests(req_file)
        name = req_file.split('\\')[-1][:-5]
        tasks.append((n,requests,name))


In [8]:
rs = []
for i in range(len(tasks)):
    if i < 89:
        continue
    problem = get_problem("nfv",tasks[i][0],tasks[i][1])
    ref_dirs = get_reference_directions("uniform", 2, n_partitions=99)
    algorithm = MOEAD(
        ref_dirs,
        n_neighbors=15,
        prob_neighbor_mating=0.7,
    )
    res = minimize(problem,
                algorithm,
                ('n_gen', 50),
                seed=1,
                save_history = True,
                verbose=True)
    rs.append(res)
    X, F = res.opt.get("X", "F")
    F = np.array(F)
    np.savetxt(f'result/moead/{tasks[i][2]}.txt',F)


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False

n_gen  |  n_eval  | n_nds  |      igd      |       gd      |       hv     
     1 |      100 |      1 |  0.6607546575 |  0.0564727092 |  0.2500000000


KeyboardInterrupt: 